# Preprocessing

In [1]:
import pandas as pd
import numpy as np


In [4]:
data = pd.read_csv("data3.csv")

In [5]:
data

,xAcc,yAcc,zAcc,xGyro,yGyro,zGyro,label
0,6.99,-0.57,-7.28,-2.75,-3.23,2.62,fall
1,6.51,-0.75,-6.93,-0.67,-6.35,4.64,fall
2,6.22,-0.63,-6.73,0.79,-5.49,3.85,fall
3,6.34,-0.62,-6.80,1.59,-2.26,0.67,fall
4,6.49,-0.39,-6.60,0.67,-0.24,-1.10,fall
...,...,...,...,...,...,...,...
96795,7.30,-1.16,-4.58,18.19,3.60,-35.77,light
96796,7.39,-0.37,-5.05,20.08,0.06,-34.55,light
96797,7.52,-1.46,-5.82,22.58,0.12,-28.02,light
96798,7.58,-2.14,-6.31,19.47,2.44,-22.52,light


### Adjust the features as we want

In [6]:
# every row is an observation of length 400 x 3 x 2

num_rows = data.shape[0]
# Set the step size to select every 400 rows
step = 400

# Create a range of indices to select rows in chunks of 400
indices = np.arange(0, num_rows, step)

# Create a list to store the transposed data
transposed_data = []
new_data = pd.DataFrame()
# Iterate over the original columns
for col in data.columns:
    if col != "label":
        # Extract values from the column and reshape into chunks of 400 rows
        column_values = data[col].values
        # reshaped_values = column_values[indices]
        # Split the list into chunks of 400 values
        chunk_size = 400
        chunks = [column_values[i:i + chunk_size] for i in range(0, len(column_values), chunk_size)]

        # Create a DataFrame
        tmp = pd.DataFrame(chunks, columns=[f'{col}_{i+1}' for i in range(chunk_size)])
        new_data = pd.concat([new_data, tmp], axis=1)

labels = np.array(data.label.iloc[np.arange(0,data.shape[0], 400)])
new_data["label"] = labels

In [6]:
# TEST PER LE SOMME DI VETTORI

# x = np.array(new_data.iloc[0, 1200:1600])
# y = np.array(new_data.iloc[0, 1600:2000])
# z = np.array(new_data.iloc[0, 2000:2400])
# s = x**2 + y**2 + z**2
# s

In [7]:
# every row is an observation of length 400 x 3 x 2

num_rows = new_data.shape[0]
# Set the step size to select every 400 rows
step = 400

# Create a range of indices to select rows in chunks of 400
indices = np.arange(0, num_rows, step)

# Create a list to store the transposed data
new_data2 = pd.DataFrame()
# Iterate over the original columns

def chunk_splitting(row):

    # Split the list into chunks of 400 values
    n_chunks = 2400/400
    chunks = np.array_split(np.array(row.values[:len(row.values)-1], dtype=float), n_chunks)

    return chunks

def acc_sum(vec):
    acc = np.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    
    return acc

def gyr_sum(vec):
    gyr = np.sqrt(vec[3]**2 + vec[4]**2 + vec[5]**2)

    return gyr

chunks = new_data.apply(lambda x: chunk_splitting(x), axis=1)

In [8]:
df = pd.DataFrame()

new_data["acc_sum"] = chunks.apply(lambda x: acc_sum(x))
new_data["gyr_sum"] = chunks.apply(lambda x: gyr_sum(x))
df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
df[[f'gyr_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.gyr_sum.to_list(), index = new_data.index)
df

C:\Users\engri\AppData\Local\Temp\ipykernel_11828\421098076.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
C:\Users\engri\AppData\Local\Temp\ipykernel_11828\421098076.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f'acc_{i+1}' for i in range(chunk_size)]] = pd.DataFrame(new_data.acc_sum.to_list(), index = new_data.index)
C:\Users\engri\AppData\Local\Temp\ipykernel_11828\421098076.py:5: PerformanceWarning

,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7,acc_8,acc_9,acc_10,...,gyr_391,gyr_392,gyr_393,gyr_394,gyr_395,gyr_396,gyr_397,gyr_398,gyr_399,gyr_400
0,10.108581,9.537688,9.185761,9.317725,9.264567,9.625679,9.470449,9.866560,9.931556,9.772226,...,1.029612,0.832226,1.108152,1.508940,1.350296,1.384558,1.316586,1.062873,0.996393,1.218113
1,9.642505,9.398622,9.307653,9.317312,9.578157,9.677939,9.672518,9.602609,9.526610,9.640954,...,1.037593,0.755910,1.080139,1.166619,1.393198,1.229349,1.508575,1.302728,1.196746,1.042785
2,9.661987,9.639725,9.577270,9.591632,9.661698,9.597072,9.501800,9.647844,9.755275,9.684782,...,44.488846,36.741091,35.978113,39.397108,45.323665,50.354353,51.365726,49.815388,50.785400,42.545700
3,9.480295,8.876773,8.682062,8.864181,8.687900,8.705142,9.638350,17.116331,10.001335,8.002706,...,8.866268,5.678010,9.178181,9.709485,7.585269,6.007537,6.188667,5.522871,6.093423,10.975719
4,9.567241,9.223112,9.656159,9.535397,9.125322,8.799966,8.890231,9.637935,15.429051,11.867860,...,25.833149,25.946726,28.139966,31.458948,28.856559,20.350327,9.901773,10.485676,20.769095,45.044041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,9.170093,9.273748,9.343463,9.500316,9.085670,9.075230,9.275139,11.338188,12.609592,10.298942,...,12.656465,24.157347,14.040239,4.396464,12.434846,38.257195,49.500611,52.183440,56.153815,51.446970
238,9.022505,9.271451,9.181166,9.263369,9.303322,9.903297,11.889706,12.140906,10.008227,8.930291,...,11.848932,16.680126,22.309641,26.740989,36.154228,52.353129,67.389355,65.728835,38.954941,21.060769
239,7.912073,9.007675,8.720384,10.116961,9.617723,10.637171,10.104375,9.309882,9.280781,9.631895,...,24.538594,27.804604,30.426965,30.604902,30.377567,24.071066,8.991780,5.798396,17.283695,27.881836
240,9.129847,8.806730,9.078882,10.509943,10.325803,9.800066,9.796203,9.374711,9.540325,9.469789,...,12.731873,14.292617,14.283893,12.658535,6.578982,7.462721,15.174574,18.183570,21.144408,29.404588


In [9]:
# little check
np.sqrt([6.99**2 + (-0.57)**2 + (-7.28)**2])

array([10.10858051])

### Feature extraction

In [14]:
from typing import List, Union
import pandas as pd
import numpy as np
import re
from scipy.fft import fft, fftfreq
from scipy.signal import periodogram
import statsmodels.api as sm
from scipy.stats import skew
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression


def maxbin(row: Union[List[float], np.ndarray], plot: bool, n_bins: int = 10) -> List[float]:
    """
    Divide the data into a specified number of bins and calculate the maximum value for each bin.

    Args:
        row (list or array): The data to be divided into bins.
        num_bins (int, optional): The number of bins to divide the data into. Defaults to 10.

    Returns:
        list: A list containing the maximum value for each bin.
    """
    n = len(row)
    freq = list(fftfreq(n*2))
    step = n//n_bins
    max_values = []
    max_freqs = []
    for i in range(0, n+1, step):
        if i==0:
            prec = i
            continue
        mm = max(row[prec: i])
        max_values.append(mm)
        max_freqs.append(freq[list(row).index(mm)])
        prec = i
    if plot:
        return max_freqs, max_values
    else:
        return max_values


def fourier_magnitudes(signal: np.ndarray, n_bins: int = 10, plot: bool = False) -> np.ndarray:
    """
    This function takes a matrix (signal) which contains a time series for each row.
    It applies the Fast Fourier Transform and uses the get_max_per_bin function to find the maximum peaks for each bin of the arrays of the power spectrum for the FFT.

    Args:
        signal (np.ndarray): The input matrix, where each row represents a time series.
        num_bins (int, optional): The number of bins to divide the data into. Defaults to 10.

    Returns:
        np.ndarray: The maximum peaks for each bin of the arrays of the power spectrum for the FFT.
    """
    signal= np.array(signal)
    # let's calculate the FFT
    magnitudes = np.apply_along_axis(fft, 0, signal) 
    # extract just the first half of power spectrum and discover the peaks
    if len(magnitudes.shape)==1:
        magnitudes=magnitudes[:((magnitudes.shape[0])//2)]
    else:
        magnitudes=magnitudes[:((magnitudes.shape[0])//2),:]
    # apply power spectrum formula
    magnitudes = np.abs(magnitudes)**2 
    # take the peaks for each bin
    peaks = np.apply_along_axis(lambda x: maxbin(x, plot, n_bins), 0, magnitudes)
    
    return peaks


# def psd_stats(signal: np.ndarray) -> np.ndarray:
#     """
#     This function takes a matrix (signal) which contains a time series for each row.
#     It calculates the Power Spectral Density (PSD) from the signal and returns three different summary statistics
#     from the distribution of PSD: the median, the mean absolute deviation, and the skewness (third moment).

#     Args:
#         signal (np.ndarray): The input matrix, where each row represents a time series.

#     Returns:
#         np.ndarray: An array containing the median, mean absolute deviation, and skewness of the Power Spectral Density.
#     """
#     signal= np.array(signal)
#     # let's calculate the Power Spectral Density
#     psd = np.array(np.abs(np.apply_along_axis(periodogram, 0, signal)[1]), dtype=np.float64)
    
#     # extract our main summaries from the distribution
#     median = np.median(psd, axis=0)
#     # calculate the third moment (skewness)
#     third_moment = skew(psd, axis=0)
#     # calculate the mean absolute deviation
#     mad = np.mean(np.abs(psd - np.mean(psd, axis=0)), axis = 0)
    
#     return np.array([median, mad, third_moment])


# def acf(signal: np.ndarray, n_lags: int = 10) -> np.ndarray:
#     """
#     This function takes a matrix (signal) which contains a time series for each row.
#     It calculates the auto-correlation for different lags.

#     Args:
#         signal (np.ndarray): The input matrix, where each row represents a time series.
#         num_lags (int, optional): The number of lags to calculate the auto-correlation for. Defaults to 10.

#     Returns:
#         np.ndarray: The auto-correlation for each time series in the input matrix.
#     """
#     signal= np.array(signal)
#     autocorrelations = np.apply_along_axis(lambda x: sm.tsa.acf(x, nlags=n_lags), 0, signal)


#     return autocorrelations

def adjust_df(s: pd.Series) -> pd.DataFrame:
    """
    This function takes a pandas Series, explodes and transposes it, and returns it as a DataFrame.

    Args:
        series (pd.Series): The input pandas Series.

    Returns:
        pd.DataFrame: The adjusted DataFrame.
    """
    df_list = [pd.DataFrame(matrix, index=[index]*len(matrix)) for index, matrix in s.items()]
    new_df = pd.concat(df_list, axis=0)
    new_df.columns = range(new_df.shape[1])
    new_df = new_df.T
    return new_df



def preproc(df, n_bins=10, n_lags=10):
    
    # group timeseries by device (accelerometer and gyroscope)
    group_dict = {}
    for col in df.columns:
        # match "acc" or "gyr"
        match = re.match("^([a-z]{3})", col)
        group_dict[col] = match.group() if match else None

    new_df = df.T.groupby(group_dict, axis=0)

    # evaluate max magnitudes (intesities of the frequencies from Fourier Fast transform) for each observation for each accelerometer
    magns = new_df.apply(lambda x: fourier_magnitudes(x, n_bins))

    # # evaluate autocorrelations
    # # autocorrs = new_df.apply(lambda x: acf(x, n_lags))
    # # evaluate psd stats
    # # psds = new_df.apply(psd_stats)

    # adjust in DataFrame format
    magns = adjust_df(magns)
    # autocorrs = adjust_df(autocorrs)
    # psds = adjust_df(psds)

    # change col names of our dataframe of magnitudes
    existing_columns = magns.columns
    # generate a list of new column names
    new_columns = []
    i=1
    for j,col_name in enumerate(existing_columns):
        if ((j+1)%(n_bins+1)) == 0:
            i=1
        new_columns.append(f"{col_name}_max_mag_{i}")
        i+=1
    magns.columns = new_columns
    new_df = magns
    # # change col names of our dataframe of autocorrelations
    # existing_columns = autocorrs.columns
    # # generate a list of new column names
    # new_columns = []
    # i=0
    # for j,col_name in enumerate(existing_columns):
    #     if ((j+1)%(n_lags+2)) == 0:
    #         i=0
    #     new_columns.append(f"{col_name}_acf_l{i}")
    #     i+=1
    # autocorrs.columns = new_columns

    # # remove all the column which correspond to autocorrelation with lag=0
    # autocorrs = autocorrs.loc[:, (autocorrs != 1).any(axis=0)]

    # # change col names of our dataframe of autocorrelations
    # existing_columns = psds.columns
    # # generate a list of new column names
    # new_columns = []
    # i=1
    # n_stats = 3
    # for j,col_name in enumerate(existing_columns):
    #     new_columns.append(f"{col_name}_psd{i}")
    #     if ((j+1)%(n_stats)) == 0:
    #         i=0
    #     i+=1
    # psds.columns = new_columns

    # # create new column 'Season' based on 'when'
    # df['when'] = pd.to_datetime(df['when'], format='%Y-%m')
    # df["Season"] = df['when'].apply(get_season)

    # # take the number of moths passed from the 'zero' time
    # min_date =  df['when'].min()
    # df['elapsed_time'] = (df['when'].dt.year - min_date.year) * 12 + (df['when'].dt.month - min_date.month)
    
    # new_df = pd.concat([df[["Season", "elapsed_time"]], magns, autocorrs, psds], axis=1)

    return new_df

#### Before

In [11]:
df

,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7,acc_8,acc_9,acc_10,...,gyr_391,gyr_392,gyr_393,gyr_394,gyr_395,gyr_396,gyr_397,gyr_398,gyr_399,gyr_400
0,10.108581,9.537688,9.185761,9.317725,9.264567,9.625679,9.470449,9.866560,9.931556,9.772226,...,1.029612,0.832226,1.108152,1.508940,1.350296,1.384558,1.316586,1.062873,0.996393,1.218113
1,9.642505,9.398622,9.307653,9.317312,9.578157,9.677939,9.672518,9.602609,9.526610,9.640954,...,1.037593,0.755910,1.080139,1.166619,1.393198,1.229349,1.508575,1.302728,1.196746,1.042785
2,9.661987,9.639725,9.577270,9.591632,9.661698,9.597072,9.501800,9.647844,9.755275,9.684782,...,44.488846,36.741091,35.978113,39.397108,45.323665,50.354353,51.365726,49.815388,50.785400,42.545700
3,9.480295,8.876773,8.682062,8.864181,8.687900,8.705142,9.638350,17.116331,10.001335,8.002706,...,8.866268,5.678010,9.178181,9.709485,7.585269,6.007537,6.188667,5.522871,6.093423,10.975719
4,9.567241,9.223112,9.656159,9.535397,9.125322,8.799966,8.890231,9.637935,15.429051,11.867860,...,25.833149,25.946726,28.139966,31.458948,28.856559,20.350327,9.901773,10.485676,20.769095,45.044041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,9.170093,9.273748,9.343463,9.500316,9.085670,9.075230,9.275139,11.338188,12.609592,10.298942,...,12.656465,24.157347,14.040239,4.396464,12.434846,38.257195,49.500611,52.183440,56.153815,51.446970
238,9.022505,9.271451,9.181166,9.263369,9.303322,9.903297,11.889706,12.140906,10.008227,8.930291,...,11.848932,16.680126,22.309641,26.740989,36.154228,52.353129,67.389355,65.728835,38.954941,21.060769
239,7.912073,9.007675,8.720384,10.116961,9.617723,10.637171,10.104375,9.309882,9.280781,9.631895,...,24.538594,27.804604,30.426965,30.604902,30.377567,24.071066,8.991780,5.798396,17.283695,27.881836
240,9.129847,8.806730,9.078882,10.509943,10.325803,9.800066,9.796203,9.374711,9.540325,9.469789,...,12.731873,14.292617,14.283893,12.658535,6.578982,7.462721,15.174574,18.183570,21.144408,29.404588


#### After

In [15]:
prova = preproc(df)
prova

,acc_max_mag_1,acc_max_mag_2,acc_max_mag_3,acc_max_mag_4,acc_max_mag_5,acc_max_mag_6,acc_max_mag_7,acc_max_mag_8,acc_max_mag_9,acc_max_mag_10,gyr_max_mag_1,gyr_max_mag_2,gyr_max_mag_3,gyr_max_mag_4,gyr_max_mag_5,gyr_max_mag_6,gyr_max_mag_7,gyr_max_mag_8,gyr_max_mag_9,gyr_max_mag_10
0,1.679561e+07,10801.345361,15443.379045,2494.648966,3506.348890,2881.347046,3416.295521,5800.178632,2368.599631,1473.222493,1.077492e+08,9.592757e+05,5.119511e+05,312838.119283,147240.538463,211720.022095,109374.244172,20964.153534,45291.483985,121592.068026
1,1.666644e+07,10751.749903,22126.371544,7530.281961,6636.895371,4383.290907,5607.831278,4917.174318,1942.169619,3949.116959,2.970916e+08,2.627339e+06,6.657685e+05,465629.751594,116254.560780,361048.330523,248070.905590,182901.528829,45836.603978,57632.006013
2,1.690417e+07,15663.913171,18483.969593,15089.006454,12469.690810,5080.274593,9184.062217,4675.716777,2361.136182,1662.668453,4.304287e+08,4.886385e+06,7.235702e+05,791065.811944,532990.905611,168062.542131,82527.888710,74912.734801,50200.552175,36780.468990
3,1.690057e+07,27834.293492,58377.754093,9126.588154,8483.198818,5839.101415,3133.863000,1640.416392,3341.617160,4192.033963,5.162937e+08,4.422935e+06,6.838762e+05,380239.911926,266260.475134,219481.118060,135248.631338,136101.761931,117920.069358,73855.265233
4,1.742747e+07,124734.902539,55886.725130,12170.351645,12544.880766,6504.401602,3993.475932,7620.558801,2822.194933,4559.995931,3.574267e+08,2.053406e+06,1.452541e+06,416519.314624,492788.710548,190757.846448,194313.119283,226042.230682,239743.219263,212045.393820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,1.638037e+07,16388.013912,11433.622947,12350.902330,15188.694745,6893.254251,3960.388446,803.969797,2660.412913,1624.898955,3.005802e+08,3.910192e+06,7.194042e+05,389663.003137,112379.522034,80732.642351,27678.196355,53746.852251,54694.151567,19737.802113
238,1.660464e+07,14302.864306,19305.212301,10808.249385,9077.769379,2102.045811,1453.138115,4566.588317,2296.186942,4651.806443,3.665159e+08,1.995195e+06,5.912486e+05,374060.032924,388659.395310,198740.942220,116755.550328,74039.095038,66162.299579,27381.467173
239,1.612340e+07,25131.303791,5905.058076,7903.859218,7405.519345,5113.755937,3755.953231,2746.683025,1416.069762,519.482796,2.961041e+08,3.504227e+06,1.211368e+06,554021.651008,124375.185208,197765.321892,96612.083350,60328.711581,36068.002933,46641.073757
240,1.613424e+07,6998.180669,15515.727474,10728.015104,4613.584971,1830.390414,812.754564,894.438233,1391.102353,1359.595058,4.060575e+08,1.856985e+06,1.246336e+06,233156.631661,205460.243089,112661.289028,127789.433541,80630.341534,72817.869024,21806.228030


# Split, scale etc... for the algorithms

In [21]:
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [62]:
# labels
y = new_data["label"]

# split data 
X_train, X_test, y_train, y_test = train_test_split(prova, y, test_size=0.2)

# scale the features (may be useful if we are going to add other features with different scale)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))

# First logistic - with all the classes (7)

In [63]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train_scaled, y_train)
preds = sgd_clf.predict(X_test_scaled)


Check for the correct classes

In [64]:
sgd_clf.classes_

array(['fall', 'lfall', 'light', 'rfall', 'sit', 'step', 'walk'],
      dtype='<U5')

In [65]:
accuracy_score(y_test, preds)

0.7959183673469388

In [66]:
confusion_matrix(y_test, preds)

array([[ 7,  1,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 1,  1, 12,  0,  2,  0,  0],
       [ 2,  0,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  8,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  1],
       [ 0,  0,  0,  0,  0,  0, 10]], dtype=int64)

# Linear classifier with RandomizedSearchCV

In [73]:
from sklearn.model_selection import GridSearchCV

In [75]:
linear_params = {"loss": ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"],
                   "penalty": ["l1", "l2", "elasticnet", None],
                   "alpha": [0.0001, 0.01, 0.1],
                   "fit_intercept": [True, False],
                   "learning_rate": ["constant", "optimal", "invscaling", "adaptive"]}

linear_search = GridSearchCV(sgd_clf, linear_params, scoring="accuracy", n_jobs=-1, cv=5)

fitted_linear_cv = linear_search.fit(X_train, y_train)

C:\Users\engri\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3240 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\engri\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\engri\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 894, in fit
    self._more_validate_params()
  File "C:\Users\engri\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_stochastic_

In [76]:
fitted_linear_cv.best_params_

{'alpha': 0.1,
 'fit_intercept': False,
 'learning_rate': 'optimal',
 'loss': 'perceptron',
 'penalty': 'l1'}

In [77]:
new_sgd_clf = fitted_linear_cv.best_estimator_
new_sgd_clf.fit(X_train_scaled, y_train)
new_preds = new_sgd_clf.predict(X_test_scaled)

I tried both RandomizedSearchCV and GridSearchCV... but both underperform the default settings... we should check for the warnings/errors when trying "illegal" parameter combo 

In [78]:
accuracy_score(y_test, new_preds)

0.7551020408163265